# Deep Learning – Classification (TensorFlow / Keras)

This notebook is part of the **ML-Methods** project.

It introduces **Deep Learning for supervised classification**
using **TensorFlow / Keras**, a high-level deep learning framework.

As with the other classification notebooks,
the first sections focus on data preparation
and are intentionally repeated.

This ensures consistency across models
and allows fair comparison of results.

-----------------------------------------------------

## Notebook Roadmap (standard ML-Methods)

1. Project setup and common pipeline  
2. Dataset loading  
3. Train-test split  
4. Feature scaling (why we do it)  

----------------------------------

5. What is this model? (Intuition)  
6. Model training  
7. Model behavior and key parameters  
8. Predictions  
9. Model evaluation  
10. When to use it and when not to  
11. Model persistence  
12. Mathematical formulation (deep dive)  
13. Final summary – Code only  

-----------------------------------------------------

## How this notebook should be read

This notebook is designed to be read **top to bottom**.

Before every code cell, you will find a short explanation describing:
- what we are about to do
- why this step is necessary
- how it fits into the overall process

Compared to PyTorch,
this notebook hides more low-level details,
allowing us to focus on **model structure and behavior**
rather than training mechanics.

-----------------------------------------------------

## What is Deep Learning (in this context)?

Deep Learning refers to models based on
**multi-layer neural networks**.

These models:
- learn complex, non-linear relationships
- automatically build internal representations
- improve performance as data complexity increases

In this notebook, we focus on:
**Deep Learning for tabular classification**
using fully connected neural networks.

-----------------------------------------------------

## Why TensorFlow / Keras?

TensorFlow / Keras provides:
- a high-level API for defining neural networks
- automatic training loops
- built-in loss functions and optimizers

This allows us to:
- write less code
- reduce implementation errors
- focus on architecture and results

Keras is ideal for:
- rapid prototyping
- production-ready models
- clean and readable implementations

-----------------------------------------------------

## Execution model: eager and graph

TensorFlow supports two execution modes:

- **Eager execution**  
  Operations are executed immediately,
  similar to standard Python code.

- **Graph execution**  
  Computation is compiled into a static graph
  for optimized performance.

Keras abstracts this complexity,
allowing us to work intuitively
while benefiting from optimized execution.

-----------------------------------------------------

## What you should expect from the results

With Deep Learning (TensorFlow / Keras), you should expect:

- performance comparable to PyTorch
- clean and compact model definitions
- faster experimentation
- less control over low-level training details

However:
- debugging is less explicit than PyTorch
- internal mechanics are more hidden
- understanding requires conceptual clarity

-----------------------------------------------------


___
## 1. Project setup and common pipeline

In this section we set up the common pipeline
used across classification models in this project.

Although this notebook uses **TensorFlow / Keras**,
the overall workflow remains identical
to the PyTorch and scikit-learn notebooks.

This ensures:
- conceptual continuity
- fair comparison across frameworks
- focus on what changes: the implementation


In [1]:
# ====================================
# Common imports used across classification models
# ====================================

import numpy as np
import pandas as pd

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay
)

from pathlib import Path
import matplotlib.pyplot as plt
import joblib

# ====================================
# TensorFlow / Keras imports
# ====================================

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


### What changes with TensorFlow / Keras

Compared to PyTorch:
- the overall pipeline remains the same
- data preparation and evaluation stay unchanged
- the training loop is handled automatically

With Keras:
- models are defined declaratively
- training is performed via `.fit()`
- gradients and updates are hidden

This reduces code complexity
but also hides some internal mechanics.

In the next section,
we will load the dataset
and prepare it for TensorFlow training.
